<a href="https://colab.research.google.com/github/davidlealo/sic_ai_2025_jun/blob/main/04pln/clase_30.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Clase 30

# Proyecto Final – PLN y Aplicaciones Interactivas en Google Colab

## Objetivo del proyecto
Desarrollar una aplicación interactiva en Google Colab que resuelva un problema real de búsqueda, clasificación o recomendación de información textual, integrando:

- Procesamiento de Lenguaje Natural (PLN).
- Técnicas de modelado vistas en el curso: TF-IDF, Word2Vec, LDA, Keras.
- Interfaz interactiva en Colab usando `from IPython.display import Javascript, display, HTML`.
- (Opcional pero muy recomendado) Uso de SQL para gestionar y consultar datos.

## Descripción general
Cada estudiante o grupo deberá:

1. Elegir un caso de uso:
   - Buscador inteligente para un catálogo.
   - Clasificador automático de documentos o productos.
   - Sistema de recomendación.

2. Seleccionar un dataset real.

3. Implementar un pipeline completo:
   - Carga y exploración de datos.
   - Preprocesamiento de texto.
   - Vectorización y modelado.
   - (Opcional) Consultas en SQL para filtrar o explorar información.
   - Interfaz interactiva en Colab.

4. Probar la aplicación con entradas del usuario.

## Integración opcional con SQL (muy recomendable)
Usar SQL permite:

- Filtrar datos antes del procesamiento.
- Guardar resultados procesados para consultas rápidas.
- Integrar consultas SQL en la interfaz interactiva.

Ejemplo:
```python
import sqlite3
import pandas as pd

# Guardar en base de datos SQLite
conn = sqlite3.connect('dataset.db')
df.to_sql('peliculas', conn, if_exists='replace', index=False)

# Consulta filtrando por género
query = "SELECT * FROM peliculas WHERE genero LIKE '%acción%'"
df_accion = pd.read_sql_query(query, conn)
```

Sugerencia: Si lo usas, documenta cómo SQL mejora tu flujo de trabajo.

## Requerimientos técnicos
- Entorno: Google Colab.
- Lenguaje: Python 3.

Librerías mínimas:
```python
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from IPython.display import Javascript, display, HTML
```

Opcionales:
- `gensim` para Word2Vec.
- `keras` / `tensorflow` para clasificación.
- `sqlite3` para SQL.
- `matplotlib` / `seaborn` para visualización.

## Casos de uso sugeridos
- Buscador inteligente para un catálogo de películas, libros o productos.
- Clasificador automático de reseñas, noticias o documentos.
- Sistema de recomendación de productos, películas, libros o artículos.
- Asistente de búsqueda académica.
- Sistema de preguntas y respuestas sobre una base textual.

## Sugerencia de datasets
- IMDb Reviews – https://ai.stanford.edu/~amaas/data/sentiment/
- Amazon Fine Food Reviews – https://www.kaggle.com/datasets/snap/amazon-fine-food-reviews
- AG News Classification – https://www.kaggle.com/datasets/amananandrai/ag-news-classification-dataset
- BBC News Dataset – https://www.kaggle.com/datasets/yufengdev/bbc-fulltext-and-category
- SMS Spam Collection – https://www.kaggle.com/datasets/uciml/sms-spam-collection-dataset
- Wikipedia Movie Plots – https://www.kaggle.com/datasets/jrobischon/wikipedia-movie-plots

## Formato de entrega
- Notebook en Colab con:
  - Código comentado.
  - Resultados visibles.
  - Interfaz interactiva.
  - (Opcional) Uso de SQL.
- Documento PDF con:
  - Resumen del proyecto.
  - Capturas de la app.
  - Explicación del modelo.

## Evaluación
| Criterio | Ponderación |
|----------|-------------|
| Preparación y limpieza de datos | 20% |
| Implementación del modelo de PLN | 25% |
| Interfaz interactiva en Colab | 25% |
| (Opcional) Uso de SQL documentado | +10% extra |
| Resultados, análisis y documentación | 20% |


## Ejemplo

In [2]:
# Librerías necesarias
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from IPython.display import HTML, display
import sqlite3  # Opcional

In [5]:
# Cargar dataset de ejemplo (películas)
# Este enlace es confiable y permite leer directamente desde GitHub
# Muy recomendable buscar otras tipo movielens o en kaggle

url = "https://gist.githubusercontent.com/tiangechen/b68782efa49a16edaf07dc2cdaa855ea/raw/movies.csv"
df = pd.read_csv(url)

# Verificamos que tenga columnas de título y descripción
# Este dataset usa 'Film' como título y 'Genre' o
# se puede reemplazar 'Genre' por alguna columna de descripción.
df = df.rename(columns={'Film': 'title', 'Genre': 'overview'})

df


,title,overview,Lead Studio,Audience score %,Profitability,Rotten Tomatoes %,Worldwide Gross,Year
0,Zack and Miri Make a Porno,Romance,The Weinstein Company,70,1.747542,64,$41.94,2008
1,Youth in Revolt,Comedy,The Weinstein Company,52,1.090000,68,$19.62,2010
2,You Will Meet a Tall Dark Stranger,Comedy,Independent,35,1.211818,43,$26.66,2010
3,When in Rome,Comedy,Disney,44,0.000000,15,$43.04,2010
4,What Happens in Vegas,Comedy,Fox,72,6.267647,28,$219.37,2008
...,...,...,...,...,...,...,...,...
72,Across the Universe,romance,Independent,84,0.652603,54,$29.37,2007
73,A Serious Man,Drama,Universal,64,4.382857,89,$30.68,2009
74,A Dangerous Method,Drama,Independent,89,0.448645,79,$8.97,2011
75,27 Dresses,Comedy,Fox,71,5.343622,40,$160.31,2008


In [ ]:
df = df[['title', 'overview']].dropna().head(1000)
df.head()

In [ ]:
# Búsqueda básica
def busqueda_basica(query):
    resultados = df[df['overview'].str.contains(query, case=False, na=False)]
    return resultados[['title', 'overview']].head(5)

busqueda_basica("space")

In [ ]:
# (Opcional) Guardar en SQL
conn = sqlite3.connect('peliculas.db')
df.to_sql('peliculas', conn, if_exists='replace', index=False)

pd.read_sql_query("SELECT * FROM peliculas WHERE title LIKE '%Star%'", conn).head()

In [ ]:
# Búsqueda mejorada con TF-IDF
vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = vectorizer.fit_transform(df['overview'])

def busqueda_tfidf(query, n_resultados=5):
    query_vec = vectorizer.transform([query])
    similitudes = cosine_similarity(query_vec, tfidf_matrix).flatten()
    indices = similitudes.argsort()[::-1][:n_resultados]
    return df.iloc[indices][['title', 'overview']]

busqueda_tfidf("space")

In [ ]:
# Comparación de resultados
def comparar_busquedas(query):
    print("Resultados búsqueda básica:")
    display(busqueda_basica(query))
    print("\nResultados búsqueda TF-IDF:")
    display(busqueda_tfidf(query))

comparar_busquedas("space")

In [ ]:
# Métricas de relevancia (simple)
def metricas_relevancia(query, metodo):
    resultados = metodo(query)
    palabras_query = set(query.lower().split())
    puntajes = []
    for desc in resultados['overview']:
        palabras_desc = set(str(desc).lower().split())
        interseccion = palabras_query.intersection(palabras_desc)
        puntaje = len(interseccion) / len(palabras_query)
        puntajes.append(puntaje)
    return np.mean(puntajes)

q = "space"
print(f"Relevancia media básica: {metricas_relevancia(q, busqueda_basica):.2f}")
print(f"Relevancia media TF-IDF: {metricas_relevancia(q, busqueda_tfidf):.2f}")

In [ ]:
# Interfaz interactiva en Colab
from google.colab import output

def mostrar_resultados(consulta):
    comparar_busquedas(consulta)
    print(f"Relevancia básica: {metricas_relevancia(consulta, busqueda_basica):.2f}")
    print(f"Relevancia TF-IDF: {metricas_relevancia(consulta, busqueda_tfidf):.2f}")

output.register_callback('notebook.buscar', mostrar_resultados)

def interfaz_busqueda():
    html_code = """
    <h3>Sistema de Búsqueda de Películas</h3>
    <input type="text" id="consulta" placeholder="Escribe tu búsqueda" style="width:300px; padding:5px;">
    <button onclick="buscar()">Buscar</button>
    <div id="resultado"></div>
    <script>
    function buscar() {
        var consulta = document.getElementById('consulta').value;
        google.colab.kernel.invokeFunction('notebook.buscar', [consulta], {});
    }
    </script>
    """
    display(HTML(html_code))

interfaz_busqueda()